# Ceci est la partie 1 des TPs d'OASIS.
Le texte du TP est dans un fichier pdf et distribué par l'enseignant.

 techniquement: Lancer la première cellule sans y réfléchir.
 Les cellules d'après sont référencées en fonction des sections du TP et vous aident à réaliser ce qui est demandé par le sujet du TP.
  


In [1]:
import numpy as np
import simpleaudio as sa
from IPython.display import Audio,display
import sys
import tempfile
import IPython
from skimage.transform import rescale
import matplotlib.pyplot as plt
import subprocess
import scipy.io as io
import scipy
# simplification de certains noms.
pi=np.pi
sin=np.sin
exp=np.exp
cos=np.cos
log = np.log

def joue_son(s,Fe=44100):
  """ Le son est normalisé et joué à la fréquence d'échantillonnage Fe qui vaut
   44100 par défaut."""
  if len(s.shape)>1:
    print("joue_son erreur: Le son n'est pas mono!")
    return
  son=s.copy().astype(np.float32)
  sm=(abs(son)).max()
  if sm==0:
    print("Attention: son uniformément nul!")
  else:
    son/=sm # normalisation
    son=son*(2**15-1)
  son=son.astype(np.uint16)
  play_obj = sa.play_buffer(son, 1, 2, Fe)
  play_obj.wait_done()


def viewimage(im, normalize=True,zoom=1,order=0,titre='',displayfilename=False):
    imin=im.copy().astype(np.float32)
    if len(im.shape)>2:
      imin = rescale(imin, zoom, order=order, channel_axis=2)
    else:
      imin = rescale(imin, zoom, order=order)
    if normalize:
        imin-=imin.min()
        if imin.max()>0:
            imin/=imin.max()
    else:
        imin=imin.clip(0,255)/255
    imin=(imin*255).astype(np.uint8)
    filename=tempfile.mktemp(titre+'.png')
    if displayfilename:
        print (filename)
    plt.imsave(filename, imin, cmap='gray')
    IPython.display.display(IPython.display.Image(filename))

def charge_image(fich):
    a=np.float32(plt.imread(fich))
    return a


def lire_fichier_son(fichier):
  f=io.wavfile.read(fichier)
  if len(f[1].shape)>1:
    out=f[1][:,0] #pas de stereo
  else:
    out=f[1]
  Fe=f[0]
  return (out,Fe) # renvoie un canal sonore (pas de   stereo) et la Freq. d'echant. Fe

#lien_donnees="https://www.telecom-paris.fr/perso/~ladjal/donnees_tp1_oasis.tgz"
fichier="donnees_tp1_oasis.tgz"
repdonnees="donnees_tp1_oasis/"



# Lecture d'un son depuis le disque

In [2]:
#exemple de lecture de son et le jouer.
#%%
fichier='handel.wav'
#fichier='oiseau.wav'
#fichier='marche.wav'

son,Fe=lire_fichier_son(repdonnees+fichier)
joue_son(son,Fe=Fe)


: 

# Écoute d'un écho

In [ ]:
# On utilise la fonction np.convolve pour convoluer le son avec un h d'écho

Fe=44100
temps_echo = 0.1 #choisir un temps d'echo, ici 0,1 seconde
#temps_echo = 0.5 #  0,5 seconde
#temps_echo = 0.8 #  0,8
pos_echo=int(temps_echo*10) # Par quoi multiplier le temps pour obtenir une position dans Z
h=np.zeros(pos_echo)
h[0]=1
h[-1]=0.8

son_avec_echo=np.convolve(son,h)
joue_son(son_avec_echo)

"""
Complexité du calcul réalisé :
On a N = pos_echo    opérations pour la convolution par terme du fichier son
Et la convolution est réalisée NT = Fe * Durée du son

On a donc au total une complexité en O(N.NT), qui dépend de t1

En prenant une transformée de Fourier Rapide de Fourier (ou FFT) en  O(NT.log(NT)), on aurait :
conv(son,h) = FFT-1 (FFT(son).FFT(h))
"""


In [2]:
# écoute d'une onde
exp=np.exp
i=1j
pi=np.pi
n=np.arange(0,100000)
f=0.01
u=exp(2*i*pi*f*n)
joue_son(np.real(u),44100)
joue_son(np.real(u),22050)



: 

In [ ]:
#Definition des filtres rejecteurs

def rejette1(entree,f0,Fe):

#valeur recommandee pour rho: au moins 0.9 et jamais plus de 1
    lfilter=scipy.signal.lfilter
    z0=exp(2*i*pi*f0/Fe); # position sur le cercle unite de la frequence a eliminer

    sortie=lfilter([1 ,-z0],1,entree.astype(np.float32));

    sortie=lfilter([1, -np.conj(z0)],1,sortie); #Le signal est reel, il faut donc eliminer -f0 aussi!


    sortie=np.real(sortie);# En raison d'erreurs d'arrondi il peut subsister une partie imaginaire
    return sortie


def rejette2(entree,f0,Fe,rho):

#valeur recommandee pour rho: au moins 0.9 et jamais plus de 1
    lfilter=scipy.signal.lfilter
    z0=exp(2*i*pi*f0/Fe); # position sur le cercle unite de la frequence a eliminer

    sortie=lfilter([1 ,-z0],[1 ,-rho*z0],entree.astype(np.float32));

    sortie=lfilter([1, -np.conj(z0)],[1 ,-rho*np.conj(z0)],sortie); #Le signal est reel, il faut donc eliminer -f0 aussi!

    sortie=np.real(sortie);# En raison d'erreurs d'arrondi il peut subsister une partie imaginaire
    return sortie

def norme2(X): # renvoie la norme 2 d'une variable. Utile pour calculer des erreurs
  return ((((X).astype(np.float32))**2).sum())**0.5




In [ ]:
# Ici on ajoute un son parasite et on ecoute
f0=1261
y=son[:4*Fe] # le son de départ doit contenir au moins 4 secondes (sinon changer)
n=np.arange(len(y)); #le temps discret que dure le signal y
sonbruite=y+0.5*(abs(y).max())*np.cos(2*pi*f0/Fe*n); #on ajoute une onde parasite
joue_son(sonbruite,Fe=Fe)



In [ ]:
# ecoute et evaluation de rejette1
sortie1=rejette1(sonbruite,f0,Fe)
joue_son(sortie1,Fe)
erreur1=norme2(sortie1-y)/norme2(y)
print ("erreur relative après rejette 1 en poucentage",erreur1*100)

In [ ]:
# ecoute et evaluation de rejette2
sortie2=rejette2(sonbruite,f0,Fe,0.999)
joue_son(sortie2,Fe)
erreur2=norme2(sortie2-y)/norme2(y)
print ("erreur relative après rejette 2 en poucentage",erreur2*100)